<a href="https://colab.research.google.com/github/Jacksstt/HelloGit/blob/main/Dynamic_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pandas
! pip install numpy
! pip install prophet


In [9]:

import os
assert os.path.exists("dataset/calendar.csv"), "データセットのダウンロードに失敗しています"
assert os.path.exists("dataset/listings.csv"), "データセットのダウンロードに失敗しています"
assert os.path.exists("dataset/reviews.csv"), "データセットのダウンロードに失敗しています"


In [ ]:
from pathlib import Path
import numpy
import pandas

OUTPUT_PATH = Path("output")
OUTPUT_PATH.mkdir(exist_ok=True)

calendar = pandas.read_csv("dataset/calendar.csv")

import datetime
# カラムの説明
# listing_id: 物件ID
# date: 日付
# available: 利用可能かどうか, 't'なら利用可能, 'f'なら利用不可能
# price: 利用価格, availableが't'の場合のみ値が入っている
def float_price(x):
    if isinstance(x, str):
        x = x.replace(",", "")
        return float(x[1:])
    elif numpy.isnan(x):
        return None
    raise

# 0が月曜日, 6が日曜日
calendar["weekday"] = calendar["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday())
calendar["price"] = calendar["price"].apply(lambda x: float_price(x))
calendar["count"] = 1
calendar.head(10)


In [ ]:
stat = calendar.groupby(by=["listing_id"])["price"].unique().reset_index()
count = calendar[calendar["available"] == "t"].groupby("listing_id")["count"].sum().reset_index()
stat["price_count"] = stat["price"].apply(lambda x: len(x))
stat = stat.merge(count, on="listing_id", how="inner")
stat.sort_values(by=["count", "price_count"], ascending=False).head(10)


In [12]:
listing_id = 1251707
target_calendar = calendar[calendar["listing_id"] == listing_id].drop(["available", "count"], axis=1)
target_calendar.to_csv("output/origin.csv", index=False)
del calendar


In [13]:
import random
random.seed(1)
# week_id: 週番号
week_id_list = []
week_id = 0
for i, row in target_calendar.iterrows():
    if row["weekday"] == 0:
        week_id += 1
    week_id_list.append(week_id)
target_calendar["week_id"] = week_id_list

# order: 注文数
order_list = []
for i, row in target_calendar.iterrows():
    week_id = row["week_id"]
    price = row["price"]
    trend = week_id * 2
    weekday_trend = 30 if row["weekday"] in [5, 6] else 0
    price_regression = (price - 150) * (-0.5)
    rand = random.random() * 30
    order = trend + weekday_trend + price_regression + rand
    order_list.append(order)
target_calendar["order"] = order_list

target_calendar.to_csv("output/target_calendar.csv", index=False)


In [14]:
from prophet import Prophet
target_calendar["ds"] = target_calendar["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
target_calendar["y"] = target_calendar["order"]


In [ ]:
train_range = 10 # 学習に使うデータ数（10週間分）
predict_range = 1 # 予測するデータ数(1週間分)
start_week_id = 1
pred_df_list = []
for i in range(40):
    end_week_id = start_week_id + train_range
    predict_week_id = end_week_id + predict_range
    # 学習に使うデータをweek_idを使って絞る
    train_df = target_calendar[target_calendar["week_id"].apply(lambda x: start_week_id <= x <= end_week_id)]
    # 予測に使うデータをweek_idを使って絞る
    predict_df = target_calendar[target_calendar["week_id"] == predict_week_id]
    model = Prophet(weekly_seasonality=True)
    model.add_regressor("price")
    # 学習
    model.fit(train_df)
    # 予測, yhatが予測値
    pred = model.predict(predict_df)[["ds", "yhat"]]
    pred_df_list.append(pred)
    start_week_id += 1


In [ ]:
total_pred_df = pandas.concat(pred_df_list, axis=0)
total_pred_df.head(10)


In [17]:
base_df = target_calendar.copy()
pred_df = base_df.merge(total_pred_df, on="ds", how="left")
pred_df.to_csv("output/pred.csv", index=False)


In [18]:
# 仮想売り上げ
# 価格×注文数(注文は100までしか捌けないものとする)
pred_df["sale"] = pred_df["price"] * pred_df["yhat"].apply(lambda x: min(x, 100))
pred_df["sale"].sum() # 3327615.547636597


3327640.1992133274

In [ ]:
from itertools import permutations
def generate_simulation_pattern(predict_df):
    # ダイナミックプライシングでシミュレーションする価格のデータを生成
    df = predict_df.copy()
    base_price = int(predict_df["price"].mean())
    pattern = [0.8, 0.9, 1.0, 1.1, 1.2]
    for weekday, holiday in permutations(pattern, 2):
        df.loc[df["weekday"] <= 4, "price"] = base_price * weekday
        df.loc[df["weekday"] >= 5, "price"] = base_price * holiday
        yield df

def calc_pred_sale(df):
    # 売上の計算
    sale = df["price"] * df["yhat"].apply(lambda x: min(x, 100))
    return sale.sum()

train_range = 10
predict_range = 1
start_week_id = 1
origin_pred_df_list = []
best_pred_df_list = []
for i in range(40):
    end_week_id = start_week_id + train_range
    predict_week_id = end_week_id + predict_range
    train_df = target_calendar[target_calendar["week_id"].apply(lambda x: start_week_id <= x <= end_week_id)]
    predict_df = target_calendar[target_calendar["week_id"] == predict_week_id]
    model = Prophet(weekly_seasonality=True)
    model.add_regressor("price")
    model.fit(train_df)

    # 元々の設定価格での予測結果
    pred = model.predict(predict_df)[["ds", "yhat"]]
    origin_pred_df_list.append(pred)

    # ダイナミックプライシングで最適な価格を計算した場合での予測結果
    best_sale = 0
    best_pred = None
    for simulated_df in generate_simulation_pattern(predict_df):
        # 価格を変化させて予測する
        pred = model.predict(simulated_df)[["ds", "yhat"]].merge(
            simulated_df[["ds", "price"]], on="ds", how="inner"
        )
        # 売上の計算
        sale = calc_pred_sale(pred)
        # 売上が改善されたらそのときの価格を記録しておく
        if best_sale <= sale:
            best_pred = pred
            best_sale = sale
    best_pred_df_list.append(best_pred)
    start_week_id += 1


In [ ]:
base_df = target_calendar.copy().rename(columns={"price": "origin_price"})
best_df = pandas.concat(best_pred_df_list, axis=0)
origin_df = pandas.concat(origin_pred_df_list, axis=0).rename(columns={"yhat": "origin_price_yhat"})
summary = base_df.merge(
    best_df, on="ds", how="left"
).merge(
    origin_df, on="ds", how="left"
)
summary["origin_order"] = summary["origin_price_yhat"].apply(lambda x: min(x, 100))
summary["origin_price_sale"] = summary["origin_price"] * summary["origin_order"]
summary["best_order"] = summary["yhat"].apply(lambda x: min(x, 100))
summary["best_price_sale"] = summary["price"] * summary["best_order"]
print(summary["origin_price_sale"].sum(), summary["best_price_sale"].sum())
summary.to_csv("output/best_pred.csv", index=False)
